## 連立一次方程式を Python の数理最適化 Libray で解く

In [1]:
import pulp  # PuLP library の取り込み

problem = pulp.LpProblem('SLE', pulp.LpMaximize)  # 数理 Model の定義。第１引数は任意の名前。第２引数は、最大化問題を解く、という指定（※今回は最大化問題を解いているわけではないのでおまじないに近い）

x = pulp.LpVariable('x', cat='Continuous')  # 変数 x を定義。pulp.LaVariable('任意の名前', cat='変数が連続変数、つまり実現値をとることを指定')
y = pulp.LpVariable('y', cat='Continuous')  # 変数 y を定義。pulp.LaVariable('任意の名前', cat='変数が連続変数、つまり実現値をとることを指定')

problem += 120 * x + 150 * y == 1440  # 連立一次方程式を定義
problem += x + y == 10  # 連立一次方程式を定義

status = problem.solve()  # 定義した数理 Model を解く。解けたか解けなかったの情報を返す

print('Status:', pulp.LpStatus[status])  # 最適化計算をした結果を表示
print('x=', x.value(), 'y=', y.value())  # x, y 計算結果の値を表示

Status: Optimal
x= 2.0 y= 8.0


## 線形化問題を Python の数理最適化 Libray で解く

In [2]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)  # PuLP の取り込み

x = pulp.LpVariable('x', cat='Continuous')  # 変数x をpulp.LpVariable('任意の名前', cat='連続変数、つまり実現値をとることを指定')
y = pulp.LpVariable('y', cat='Continuous')  # 変数y をpulp.LpVariable('任意の名前', cat='連続変数、つまり実現値をとることを指定')

problem += 1*x + 3*y <= 30  # 線形問題の条件式を定義
problem += 2*x + 1*y <= 40  # 線形問題の条件式を定義
problem += x >= 0  # 線形問題の条件式を定義
problem += y >= 0  # 線形問題の条件式を定義

problem += x + 2*y  # 数理 Model: problem の目的関数を定義
# LpProblem() は、+= 演算子の右辺が制約であれば制約式を追加し、関数であれば目的関数を追加する

status = problem.solve()  # 定義した数理 Model を解く

print('Status:', pulp.LpStatus[status])  # 解けたか解けなかったか結果を表示
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())  # x, y と最大化しようとした利得の関数値を表示

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0
